# Parte 2 – Modelagem e Avaliação (Machine Learning)

Neste trabalho serão desenvolvidas as etapas do pipeline de ciência de dados, relacionado ao treinamento de modelos de Aprendizado de Máquina. Sendo utilizado o mesmo dataset da Parte 1.


## 1. Importação e carregamento com KaggleHub

In [4]:
# Instalar as dependências (executar apenas uma vez no ambiente)
# !pip install kagglehub[pandas-datasets]

import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Definição do caminho do arquivo dentro do dataset do Kaggle.
# No Kaggle, esse dataset do Rodolfo Saldanha possui um arquivo chamado "marketing_campaign.csv".
file_path = "marketing_campaign.csv"

# Carregamento da versão mais recente do dataset diretamente do Kaggle
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "rodsaldanha/arketing-campaign",  # slug do dataset no Kaggle
    file_path,
    pandas_kwargs={"sep": ";"}
)

# Visualização das 5 primeiras linhas
df.head()

/tmp/ipython-input-2122527834.py:13: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'arketing-campaign' dataset.


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


## 2. Descrição e motivação do problema

Na Parte 1 foi realizada a Análise Exploratória de Dados (EDA) utilizando o dataset **"Marketing Campaign"** do Kaggle, que traz informações de clientes de uma empresa (idade, escolaridade, estado civil, renda, gastos em diferentes tipos de produtos, resposta a campanhas de marketing etc.).

Na Parte 2, o foco passa a ser a aplicação de técnicas de Aprendizado de Máquina. O problema foi formulado como uma tarefa de **classificação binária**, em que o objetivo é prever se o cliente irá aceitar ou não uma campanha de marketing.

A variável alvo é definida como:

- `Response = 1` → o cliente aceitou a última campanha;
- `Response = 0` → o cliente não aceitou a última campanha.

Esse problema é relevante porque, ao prever quais clientes têm maior probabilidade de aceitar a campanha, a empresa pode direcionar melhor os esforços de marketing, economizar recursos e aumentar a taxa de conversão.

> Com isso, é atendido o critério: “O que será previsto/classificado? Por que isso é relevante dentro do contexto do dataset?”


## 3. Resumo rápido do dataset

In [19]:
# Tipos de dados
df.dtypes

# Distribuição da variável alvo (Response)
df['Response'].value_counts(normalize=True)

,proportion
Response,
0,0.850893
1,0.149107


## 4. Resumo do dataset

- Fonte: Kaggle – *"Marketing Campaign"* (Rodolfo Saldanha).
- Cada linha corresponde a um cliente.
- Algumas colunas importantes:
  - `Year_Birth`: ano de nascimento do cliente;
  - `Education`: nível de escolaridade;
  - `Marital_Status`: estado civil;
  - `Income`: renda anual estimada;
  - `MntWines`, `MntFruits`, `MntMeatProducts`, etc.: gastos em diferentes tipos de produto;
  - `Kidhome`, `Teenhome`: quantidade de crianças/adolescentes em casa;
  - `Recency`: dias desde a última compra;
  - `AcceptedCmp1` a `AcceptedCmp5`: aceitação de campanhas anteriores;
  - `Complain`: indica se houve reclamação nos últimos 2 anos;
  - `Response`: aceitação da campanha atual (alvo deste trabalho).

A distribuição da variável `Response` é desbalanceada: a maioria dos clientes não aceita a campanha (`Response = 0`) e uma minoria aceita (`Response = 1`).

## 5. Pré-processamento para Machine Learning

Nesta etapa é repetido o pré-processamento, agora partindo do `df` carregado com o KaggleHub.

In [6]:
# Cópia do dataframe original para o fluxo de Machine Learning
df_ml = df.copy()

# Remoção de linhas com renda nula, pois Income é uma variável importante
df_ml = df_ml.dropna(subset=['Income'])

# Criação da variável de idade aproximada
df_ml['Age'] = 2025 - df_ml['Year_Birth']

# Criação da variável Total_Spent somando os gastos dos principais produtos
mnt_cols = [
    'MntWines', 'MntFruits', 'MntMeatProducts',
    'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'
]
df_ml['Total_Spent'] = df_ml[mnt_cols].sum(axis=1)

# Criação da variável Children com o total de filhos/adolescentes em casa
df_ml['Children'] = df_ml['Kidhome'] + df_ml['Teenhome']

# Remoção de colunas que não contribuem para o modelo ou são identificadores
cols_to_drop = ['ID', 'Year_Birth', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue']
df_ml = df_ml.drop(columns=cols_to_drop, errors='ignore')

df_ml.head()

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Total_Spent,Children
0,Graduation,Single,58138.0,0,0,58,635,88,546,172,...,0,0,0,0,0,0,1,68,1617,0
1,Graduation,Single,46344.0,1,1,38,11,1,6,2,...,0,0,0,0,0,0,0,71,27,2
2,Graduation,Together,71613.0,0,0,26,426,49,127,111,...,0,0,0,0,0,0,0,60,776,0
3,Graduation,Together,26646.0,1,0,26,11,4,20,10,...,0,0,0,0,0,0,0,41,53,1
4,PhD,Married,58293.0,1,0,94,173,43,118,46,...,0,0,0,0,0,0,0,44,422,1


## 6. Explicação do pré-processamento

Para preparar os dados para o treinamento dos modelos, foram realizados os seguintes passos:

1. **Remoção de nulos em `Income`**: a renda é uma variável importante no contexto de marketing, de modo que as linhas com valores ausentes nessa coluna foram removidas.
2. **Criação de novas variáveis (feature engineering)**:
   - `Age`: idade aproximada do cliente em 2025 (`2025 - Year_Birth`);
   - `Total_Spent`: soma dos gastos nos 6 tipos de produtos (`MntWines`, `MntFruits`, `MntMeatProducts`, `MntFishProducts`, `MntSweetProducts`, `MntGoldProds`);
   - `Children`: número total de filhos/adolescentes em casa (`Kidhome + Teenhome`).
3. **Remoção de colunas**:
   - `ID`: identificador do cliente, sem conteúdo preditivo relevante;
   - `Year_Birth`: informação já utilizada para gerar `Age`;
   - `Dt_Customer`, `Z_CostContact`, `Z_Revenue`: colunas não essenciais para este modelo e removidas para simplificação.

Esses passos deixam a base mais adequada para a aplicação dos algoritmos de Machine Learning.

## 7. Separação de variáveis (x, y) e tipos

---



In [7]:
# Definição da variável alvo
target = 'Response'

# X = todas as colunas exceto o alvo
X = df_ml.drop(columns=[target])
y = df_ml[target]

# Identificação de colunas numéricas e categóricas para o pré-processamento
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

numeric_features, categorical_features

(['Income',
  'Kidhome',
  'Teenhome',
  'Recency',
  'MntWines',
  'MntFruits',
  'MntMeatProducts',
  'MntFishProducts',
  'MntSweetProducts',
  'MntGoldProds',
  'NumDealsPurchases',
  'NumWebPurchases',
  'NumCatalogPurchases',
  'NumStorePurchases',
  'NumWebVisitsMonth',
  'AcceptedCmp3',
  'AcceptedCmp4',
  'AcceptedCmp5',
  'AcceptedCmp1',
  'AcceptedCmp2',
  'Complain',
  'Age',
  'Total_Spent',
  'Children'],
 ['Education', 'Marital_Status'])

A variável alvo (`Response`) foi separada das demais colunas. As colunas numéricas e categóricas também foram divididas, uma vez que recebem tratamentos distintos na etapa de pré-processamento (normalização para as numéricas e one-hot encoding para as categóricas).

## 8. Técnicas de Aprendizado de Máquina utilizadas

Foram escolhidas três técnicas de classificação para comparação:

1. **Regressão Logística**
   - Modelo linear que estima a probabilidade de um exemplo pertencer à classe positiva (`Response = 1`);
   - Utiliza uma função sigmoide para transformar a combinação linear das variáveis em uma probabilidade entre 0 e 1;
   - Frequentemente usada como baseline, além de ser relativamente fácil de interpretar.

2. **Random Forest**
   - Ensemble formado por diversas árvores de decisão;
   - Cada árvore é treinada em um subconjunto dos dados, e a decisão final é tomada por votação;
   - Consegue capturar relações não lineares e interações entre variáveis.

3. **K-Nearest Neighbors (KNN)**
   - Método baseado em vizinhança: para classificar um novo ponto, o algoritmo procura os *k* vizinhos mais próximos nos dados de treino;
   - A classe mais frequente entre esses vizinhos é utilizada como predição;
   - Depende bastante da normalização das variáveis, pois utiliza medidas de distância.

Dessa forma, são atendidos os requisitos de explicitar quais técnicas foram utilizadas e seu funcionamento geral.

## 9. Configuração dos experimentos

### 9.1 Pré-processamento com `ColumnTransformer`

In [8]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Pipeline para as variáveis numéricas: padronização (média 0, desvio padrão 1)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline para as variáveis categóricas: one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinação das duas transformações em um ColumnTransformer
preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

### 9.2 Divisão treino/teste

In [9]:
from sklearn.model_selection import train_test_split

# Divisão da base: 70% treino, 30% teste, mantendo a proporção das classes (stratify)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

As principais decisões de configuração foram:

- Utilizar **70% dos dados para treino** e **30% para teste**, com `stratify=y` para manter a proporção de clientes que aceitam ou não a campanha em ambas as partições;
- Normalizar as variáveis numéricas com `StandardScaler`, favorecendo o desempenho de algoritmos sensíveis à escala, como o KNN;
- Aplicar `OneHotEncoder` às variáveis categóricas (`Education`, `Marital_Status` etc.);
- Utilizar um `ColumnTransformer` para combinar essas transformações em um único passo dentro do pipeline.

## 10. Treinamento dos modelos e cálculo de métricas

### 10.1 Definição dos modelos

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Definição dos três modelos a serem comparados
models = {
    "LogReg": LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=42
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        class_weight='balanced'
    ),
    "KNN": KNeighborsClassifier(
        n_neighbors=5
    )
}

# O parâmetro class_weight='balanced' foi utilizado na Regressão Logística
# e na Random Forest para compensar o desbalanceamento da classe Response.


### 10.2 Treinamento, predição e cálculo das métricas

In [16]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score

results = {}

for name, clf in models.items():
    # Pipeline completo: pré-processamento + modelo
    pipe = Pipeline(steps=[
        ('preprocess', preprocess),
        ('model', clf)
    ])

    # Treinamento do modelo
    pipe.fit(X_train, y_train)

    # Predições no conjunto de teste
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]  # probabilidade da classe 1

    # Relatório de classificação
    report = classification_report(y_test, y_pred, output_dict=True)

    # ROC-AUC
    roc = roc_auc_score(y_test, y_proba)

    # Armazenamento das principais métricas da classe positiva (Response = 1)
    results[name] = {
        "Accuracy": report['accuracy'],
        "Precision (1)": report['1']['precision'],
        "Recall (1)": report['1']['recall'],
        "F1 (1)": report['1']['f1-score'],
        "ROC-AUC": roc
    }

# Transformar o dicionário em DataFrame
results_df = pd.DataFrame(results).T   # .T → modelos como linhas
results_df_rounded = results_df.round(3)

results_df_rounded


,Accuracy,Precision (1),Recall (1),F1 (1),ROC-AUC
LogReg,0.829,0.461,0.83,0.593,0.907
RandomForest,0.878,0.806,0.25,0.382,0.874
KNN,0.872,0.636,0.35,0.452,0.792


### 10.3 Validação cruzada (ROC-AUC)

In [18]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = {}

for name, clf in models.items():
    pipe = Pipeline(steps=[
        ('preprocess', preprocess),
        ('model', clf)
    ])

    scores = cross_val_score(pipe, X, y, cv=cv, scoring='roc_auc')

    cv_results[name] = {
        "ROC-AUC (mean)": scores.mean(),
        "ROC-AUC (std)": scores.std()
    }

# Transformar em tabela
cv_results_df = pd.DataFrame(cv_results).T
cv_results_df_rounded = cv_results_df.round(3)

cv_results_df_rounded

,ROC-AUC (mean),ROC-AUC (std)
LogReg,0.879,0.020
RandomForest,0.873,0.018
KNN,0.773,0.035


A validação cruzada estratificada complementa a avaliação em *hold-out*, fornecendo uma medida mais robusta da capacidade de generalização dos modelos, como solicitado nos critérios do projeto.

## 11. Análise dos resultados

Resumo das métricas mais importantes (classe 1 = quem aceita a campanha):

- **Regressão Logística**
  - Accuracy ≈ 0,83
  - Precision (classe 1) ≈ 0,46
  - Recall (classe 1) ≈ 0,83
  - F1 (classe 1) ≈ 0,59
  - ROC-AUC ≈ 0,91

- **Random Forest**
  - Accuracy ≈ 0,88
  - Precision (classe 1) ≈ 0,81
  - Recall (classe 1) ≈ 0,25
  - F1 (classe 1) ≈ 0,38
  - ROC-AUC ≈ 0,87

- **KNN (k=5)**
  - Accuracy ≈ 0,87
  - Precision (classe 1) ≈ 0,64
  - Recall (classe 1) ≈ 0,35
  - F1 (classe 1) ≈ 0,45
  - ROC-AUC ≈ 0,79

A Random Forest obteve a maior acurácia geral, porém com **recall** muito baixo para a classe positiva (≈ 0,25), o que indica que o modelo acerta bastante a classe 0, mas deixa passar muitos clientes que realmente aceitariam a campanha.

O KNN apresentou um desempenho intermediário, com boa acurácia e recall um pouco maior que o da Random Forest, mas ainda abaixo da Regressão Logística. O ROC-AUC do KNN também foi o menor entre os três modelos.

A **Regressão Logística** mostrou o melhor equilíbrio geral:
- recall elevado para a classe 1 (≈ 0,83), conseguindo identificar a maior parte dos clientes que aceitam a campanha;
- maior ROC-AUC (≈ 0,91), indicando boa separação entre clientes propensos a aceitar e a recusar a campanha.

Como o objetivo do problema é **identificar clientes com alta probabilidade de aceitar a campanha**, o recall da classe 1 é especialmente importante. Por esse motivo, a Regressão Logística se destaca como modelo mais adequado para o contexto de marketing analisado.

## 12. Conclusões

Na Parte 2 foram realizadas as seguintes etapas principais:

1. Formulação do problema como tarefa de classificação binária, visando prever a variável `Response` (aceitação da campanha).
2. Preparação dos dados:
   - remoção de nulos em `Income`;
   - criação das variáveis `Age`, `Total_Spent` e `Children`;
   - remoção de colunas de baixa relevância para o modelo.
3. Separação dos dados em treino (70%) e teste (30%), com estratificação pelas classes.
4. Aplicação de normalização nas variáveis numéricas e one-hot encoding nas variáveis categóricas.
5. Treinamento e comparação de três modelos de classificação:
   - Regressão Logística;
   - Random Forest;
   - KNN.
6. Avaliação dos modelos com métricas de classificação (accuracy, precision, recall, F1) e ROC-AUC, complementada por validação cruzada estratificada.

Entre os modelos testados, a **Regressão Logística** apresentou o melhor equilíbrio entre as métricas, com destaque para o recall da classe positiva e o valor de ROC-AUC. Assim, esse modelo é o mais indicado como candidato final para apoiar a empresa na escolha de quais clientes devem receber futuras campanhas de marketing.